1、普通attention 560

2、普通ESIM 569

3、加入full pretrain的ESIM 5779  加大deep到450 5796  HIDDEN_DIM = 200 DEEP_LAYERS = [500] 5811


In [13]:
# dic = torch.load("../datasets/models/KFold/ESIM_fold_30.5847.pth")
# dic2 = torch.load("../datasets/models/KFold/ESIM_fold_40.5779.pth")
# dic3 = torch.load("../datasets/models/KFold/ESIM_fold_70.5780.pth")
# for k,v in dic.items():
#     dic[k] = v.cpu()
# for k,v in dic2.items():
#     dic2[k] = v.cpu()
# for k,v in dic3.items():
#     dic3[k] = v.cpu()
# torch.save(dic, '../datasets/models/KFold/esim_fold_3_5847_cpu.pth')
# torch.save(dic2, '../datasets/models/KFold/esim_fold_4_5779_cpu.pth')
# torch.save(dic3, '../datasets/models/KFold/esim_fold_7_5780_cpu.pth')

In [10]:
# import pandas as pd
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import train_test_split
# train = pd.read_csv('../datasets/atec_nlp_sim_train.csv',header=None,delimiter='\t',names=['id','txt1','txt2','label'],encoding='utf-8')
# train_add = pd.read_csv('../datasets/atec_nlp_sim_train_add.csv',header=None,delimiter='\t',names=['id','txt1','txt2','label'], encoding='utf-8')
# train = pd.concat([train, train_add])
# split = StratifiedShuffleSplit(random_state=2018, test_size=0.16)

# y = train.label.values
# for tra_index, val_index in split.split(train, y):
#     real_train = train.iloc[tra_index]
#     real_valid = train.iloc[val_index]
    
# real_train, real_valid = train_test_split(train, test_size=0.19, random_state=2018)
# real_train.to_csv('../datasets/train_random.csv', index=False, encoding='utf-8')
# real_valid.to_csv('../datasets/valid_random.csv', index=False, encoding='utf-8')

In [1]:
import sys
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext import data
from time import time
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# hyper parameters
STR_MAXLEN = 30
BATCH_SIZE = 256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EMBED_DIM = 300
HIDDEN_DIM = 200
DEEP_LAYERS = [500]
LEARNING_RATE = 1e-3*0.95
EPOCHES = 20
DECAY_STEP = 2
DECAY_GAMMA = 0.99
CLASS_WEIGHT = [0.6116, 2.7397]
def print_flush(data, args=None):
    if args == None:
        print(data)
    else:
        print(data, args)
    sys.stdout.flush()
    
def pad_seq(seq, max_length):
    length = len(seq[0])
    pad_leng = 0 if length > max_length else (max_length-length)
    if pad_leng == 0:
        seq = seq[:, :max_length]
    else:
        seq = torch.cat([seq, torch.ones(len(seq), pad_leng).long()], dim=1)
    return seq

def get_mask(txt1, txt2):
    txt1 = txt1.cpu().numpy()
    txt2 = txt2.cpu().numpy()
    len_txt1 = len(txt1[0])
    len_txt2 = len(txt2[0])
    mask_txt1 = np.zeros(shape=[len(txt1), len_txt1])
    mask_txt2 = np.zeros(shape=[len(txt2), len_txt2])
    for i in range(len(txt1)):
        mask_len1 = np.where(txt1[i] == 1)[0]
        mask_len2 = np.where(txt2[i] == 1)[0]
        if len(mask_len1) == 0:
            mask_txt1[i, :] = 1
        else:
            mask_txt1[i, :mask_len1[0]] = 1
        if len(mask_len2) == 0:
            mask_txt2[i, :] = 1
        else:
            mask_txt2[i, :mask_len2[0]] = 1
    return torch.tensor(mask_txt1).float().to(DEVICE), torch.tensor(mask_txt2).float().to(DEVICE)

def wordlist_to_matrix(pretrain_path, wordlist, device, dim=300):
    word_vec = {}
    with open(pretrain_path, encoding='utf-8') as fr:
        for line in fr:
            line = line.split(' ')
            word = line[0]
            vec = line[1:]
            word_vec[word] = np.array(vec, dtype=float)
    word_vec_list = []
    oov = 0
    oov_words = []
    for idx, word in enumerate(wordlist):
        try:
            vector = np.array(word_vec[word], dtype=float).reshape(1,dim)
        except:
            oov += 1
            oov_words.append(word)
            # print(word)
            vector = np.random.rand(1, dim)
        word_vec_list.append(torch.from_numpy(vector))
    wordvec_matrix = torch.cat(word_vec_list)
    print("Load embedding finished.")
    print("Total words count: {}, oov count: {}.".format(wordvec_matrix.size()[0], oov))
    return wordvec_matrix if device == -1 else wordvec_matrix.to(device)

# torch.manual_seed(666)

In [2]:
# prepare data

def tokenizer(text): # create a tokenizer function
    return [txt for txt in text]
class BatchWrapper:
    def __init__(self, dl, iter_columns):
        self.dl, self.iter_columns = dl, iter_columns  # we pass in the list of attributes for x &amp;amp;amp;amp;lt;g class="gr_ gr_3178 gr-alert gr_spell gr_inline_cards gr_disable_anim_appear ContextualSpelling ins-del" id="3178" data-gr-id="3178"&amp;amp;amp;amp;gt;and y&amp;amp;amp;amp;lt;/g&amp;amp;amp;amp;gt;

    def __iter__(self):
        for batch in self.dl:
            yield (getattr(batch, attr) for attr in self.iter_columns)

    def __len__(self):
        return len(self.dl)
    
print_flush('process raw data...')
TEXT = data.Field(sequential=True, use_vocab=True, eos_token='<EOS>', init_token='<BOS>',pad_token='<PAD>', 
                  batch_first=True, tokenize=tokenizer)
LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)

tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("txt1", TEXT), ("txt2", TEXT),
                 ("label", LABEL)]

train = data.TabularDataset(path='../datasets/train_random.csv', format='csv', skip_header=True, fields=tv_datafields)
valid = data.TabularDataset(path='../datasets/valid_random.csv', format='csv', skip_header=True, fields=tv_datafields)

TEXT.build_vocab(train, valid, min_freq=3)
print_flush('Building vocabulary Finished.')

matrix = wordlist_to_matrix('../datasets/pretrain_embedding/pretrain_full_emb.txt', TEXT.vocab.itos, DEVICE)
# matrix = wordmodel_to_matrix('../datasets/pretrain_embedding/raw_embedding_300d.bin', TEXT.vocab.itos, DEVICE)

process raw data...
Building vocabulary Finished.
Load embedding finished.
Total words count: 1517, oov count: 8.


In [33]:
len(TEXT.vocab)

1517

In [3]:
train_iter = data.BucketIterator(dataset=train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.txt1) + len(x.txt2), shuffle=False, device=-1, repeat=False)
valid_iter = data.Iterator(dataset=valid, batch_size=BATCH_SIZE, device=-1, shuffle=False, repeat=False)

train_dl = BatchWrapper(train_iter, ["txt1", "txt2", "label"])
valid_dl = BatchWrapper(valid_iter, ["txt1", "txt2", "label"])

print_flush('prepare data done!')

prepare data done!


In [35]:
def cosine_similarity(x1, x2):
    '''compute cosine similarity between x1 and x2
    Parameters
    ----------
    x1, x2 : 2-D torch Tensor
        size (batch_size, 1)
    Returns
    -------
    distance : 2-D torch Tensor
        similarity result of size (batch_size, 1)
    '''
    return F.cosine_similarity(x1, x2).unsqueeze(1)
def attention_matrix(x1, x2, eps=1e-6):
    '''compute attention matrix using match score
    
    1 / (1 + |x · y|)
    |·| is euclidean distance
    Parameters
    ----------
    x1, x2 : 4-D torch Tensor
        size (batch_size, 1, sentence_length, h)
    
    Returns
    -------
    output : 3-D torch Tensor
        match score result of size (batch_size, sentence_length(for x2), sentence_length(for x1))
    '''
    eps = torch.tensor(eps).to(DEVICE)
    one = torch.tensor(1.).to(DEVICE)
    euclidean = (torch.pow(x1 - x2.permute(0, 2, 1, 3), 2).sum(dim=3) + eps).sqrt()
    return (euclidean + one).reciprocal()



In [5]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from time import time

class LSTM_INTERACTIVE_ATTENTION(nn.Module):
    def __init__(self, vocab, embed_dim, hidden_dim, deep_layers, pretrain_embed=torch.tensor([]), is_batch_norm=False, is_drop_out=False):
        super(LSTM_INTERACTIVE_ATTENTION, self).__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.deep_layers = deep_layers
        self.is_batch_norm = is_batch_norm
        self.is_drop_out = is_drop_out
        self.word_embed = nn.Embedding(len(vocab), embed_dim, padding_idx=vocab.stoi['<PAD>'])
        if len(pretrain_embed) != 0:
            self.word_embed.weight.data.copy_(pretrain_embed)
#             self.word_embed.weight.requires_grad = False

        self.lstm_embed = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
#         self.attention = Attention(2*hidden_dim, hidden_dim)
        self.lstm_fusion = nn.LSTM(2 * hidden_dim * 4, hidden_dim*2, batch_first=True, bidirectional=True)
        self.linear_1 = nn.Linear(2*hidden_dim*4, deep_layers[0])
#         glorot = np.sqrt(6.0 / (hidden_dim*4 + self.deep_layers[0]))
#         self.linear_1.weight.data.uniform_(-glorot, glorot)
        if self.is_batch_norm:
            self.batch_norm_1 = nn.BatchNorm1d(deep_layers[0])
        if self.is_drop_out:
            self.linear_1_dropout = nn.Dropout(0.1)
        for i, h in enumerate(self.deep_layers[1:], 1):
            setattr(self,'linear_'+str(i+1), nn.Linear(self.deep_layers[i-1], self.deep_layers[i]))
            if self.is_batch_norm:
                setattr(self, 'batch_norm_' + str(i + 1), nn.BatchNorm1d(deep_layers[i]))
            if self.is_drop_out:
                setattr(self, 'linear_'+str(i+1)+'_dropout', nn.Dropout(0.8))
        self.deep_out = nn.Linear(deep_layers[-1], 2)
#         self.attention_feature_layer = nn.Linear(STR_MAXLEN, hidden_dim*2)
#         self.deep_dropout_1 = nn.Dropout(0.5)
    def forward(self, txt1, txt2, x1_mask, x2_mask, hidden=None):
        embed_txt1 = self.word_embed(txt1)
        embed_txt2 = self.word_embed(txt2)
#         lstm_txt1 = torch.sum(self.lstm_embed(embed_txt1, hidden)[0], 1)
#         lstm_txt2 = torch.sum(self.lstm_embed(embed_txt2, hidden)[0], 1)
        lstm_txt1 = self.lstm_embed(embed_txt1, None)[0]* x1_mask[:, :, None]
        lstm_txt2 = self.lstm_embed(embed_txt2, None)[0]* x2_mask[:, :, None]
#         lstm_txt1 = lstm_txt1 
#         lstm_txt2 = lstm_txt2 
        
        weight_matrix = torch.bmm(lstm_txt1, lstm_txt2.permute(0, 2, 1))
        weight_matrix_1 = torch.exp(weight_matrix - weight_matrix.max(2, keepdim=True)[0])
        weight_matrix_2 = torch.exp(weight_matrix - weight_matrix.max(1, keepdim=True)[0])
    
        alphas = weight_matrix_1 / weight_matrix_1.sum(2, keepdim=True)
        beta = weight_matrix_2 / weight_matrix_2.sum(1, keepdim=True)
        alphas = alphas * x1_mask[:, :, None]
        beta = beta * x2_mask[:, None, :]
        
        atten_txt1to2 = torch.bmm(alphas, lstm_txt2)
        atten_txt2to1 = torch.bmm(beta.permute(0, 2, 1), lstm_txt1)
        
#         atten_txt1to2, atten_txt2to1 = self.attention(lstm_txt1, lstm_txt2)
        
        txt_substract1 = torch.abs(lstm_txt1-atten_txt1to2)
        txt_substract2 = torch.abs(lstm_txt2-atten_txt2to1)
        
        txt_multiply1 = torch.mul(lstm_txt1, atten_txt1to2)
        txt_multiply2 = torch.mul(lstm_txt2, atten_txt2to1)
        
        out_txt1 = torch.cat([lstm_txt1, atten_txt1to2, txt_substract1, txt_multiply1], 2)
        out_txt2 = torch.cat([lstm_txt2, atten_txt2to1, txt_substract2, txt_multiply2], 2)
        lstm_fusion_txt1 = self.lstm_fusion(out_txt1, None)[0]
        lstm_fusion_txt2 = self.lstm_fusion(out_txt2, None)[0]
        lstm_fusion_txt1 = lstm_fusion_txt1.unsqueeze(1)
        lstm_fusion_txt2 = lstm_fusion_txt2.unsqueeze(1)
        lstm_txt1_max = F.max_pool2d(lstm_fusion_txt1, (len(txt1[0]), 1)).squeeze()
        lstm_txt2_max = F.max_pool2d(lstm_fusion_txt2, (len(txt2[0]), 1)).squeeze() 
#         lstm_txt1_max = F.max_pool2d(lstm_txt1, (len(txt1[0]), 1)).squeeze()
#         lstm_txt2_max = F.max_pool2d(lstm_txt2, (len(txt2[0]), 1)).squeeze() 
        
        
#         lstm_txt1 = lstm_txt1.squeeze()
#         lstm_txt2 = lstm_txt2.squeeze()
        
        output = self.linear_1(torch.cat([lstm_txt1_max, lstm_txt2_max], 1))
        if self.is_batch_norm:
            output = self.batch_norm_1(output)
        output = F.relu(output)
        if self.is_drop_out:
            output = self.linear_1_dropout(output)
        for i in range(1, len(self.deep_layers)):
            output = getattr(self, 'linear_' + str(i + 1))(output)
            if self.is_batch_norm:
                output = getattr(self, 'batch_norm_' + str(i + 1))(output)
            output = F.relu(output)
            if self.is_drop_out:
                output = getattr(self, 'linear_' + str(i + 1) + '_dropout')(output)
        output = self.deep_out(output)
        return F.log_softmax(output)


In [6]:
model = LSTM_INTERACTIVE_ATTENTION(TEXT.vocab, EMBED_DIM, HIDDEN_DIM, DEEP_LAYERS, matrix)
model.to(DEVICE)
parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
criterion = nn.NLLLoss(weight=torch.tensor(CLASS_WEIGHT).float().to(DEVICE))
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)
# scheduler = StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)

In [7]:
print_every = 50
best_state = None
max_metric = 0
# model.to(DEVICE)
def predict(model, data_dl, loss_func, device):
    model.eval()
    res_list = []
    label_list = []
    loss = 0
    for text1, text2, label in data_dl:
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
        x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2, x1_mask, x2_mask)
        loss += loss_func(y_pred, label).data.cpu()
        y_pred = y_pred.data.max(1)[1].cpu().numpy()
        res_list.extend(y_pred)
        label_list.extend(label.data.cpu().numpy())
    acc = accuracy_score(res_list, label_list)
    Precision = precision_score(res_list, label_list)
    Recall = recall_score(res_list, label_list)
    F1 = f1_score(res_list, label_list)
    
    return loss, (acc, Precision, Recall, F1)
        
def evaluate(model, txt1, txt2, x1_mask, x2_mask, y):
    pred = model(txt1, txt2, x1_mask, x2_mask)
    out_batch = pred.data.max(1)[1].cpu().numpy()
    F1 = f1_score(out_batch, y)
    return F1

def training_termination(valid_result):
    if len(valid_result) >= 4:
        if valid_result[-1] < valid_result[-2] and \
            valid_result[-2] < valid_result[-3] and \
            valid_result[-3] < valid_result[-4]:
            return True
    return False

valid_iter.create_batches()
valid_batch_num = len(list(valid_iter.batches))
print_flush('start train...')
train_iter.create_batches()
batch_num = len(list(train_iter.batches))
print_flush('batch number %d '%batch_num)
valid_result = []
for epoch in range(EPOCHES):
    for param_group in optimizer.param_groups:
        print('learning rate: %.6f'% param_group['lr'])
    epoch_begin = time()
    total_loss = 0.0
    train_iter.init_epoch()
    batch_count = 0
    batch_begin_time = time()
    for text1, text2, label in train_dl:
        model.train()
        text1 = text1.to(DEVICE)
        text2 = text2.to(DEVICE)
        label = label.to(DEVICE)
        x1_mask, x2_mask = get_mask(text1, text2)
        y_pred = model(text1, text2, x1_mask, x2_mask)
        loss = criterion(y_pred, label)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        batch_count += 1
        if batch_count % print_every == 0:
            metric = evaluate(model.eval(), text1, text2, x1_mask, x2_mask, label)
            print_flush('[%d %d] loss: %.6f metric: %.6f time: %.1f s' %
                  (epoch + 1, batch_count, total_loss / print_every, metric, time() - batch_begin_time))
            total_loss = 0.0
            batch_begin_time = time()
#     scheduler.step()
    print_flush("Evaluating....")
    loss, (acc, Precision, Recall, F1) = predict(model, valid_dl, criterion, DEVICE)
    valid_result.append(F1)
    print_flush('*'*60)
    print_flush('[epoch %d]. loss: %.6f acc: %.6f f1: %.6f time: %.1f s'%(epoch+1, loss/valid_batch_num, acc, F1, time()-epoch_begin))
    print_flush('*'*60)
    if F1 > max_metric:
        best_state = model.state_dict()
        max_metric = F1
        print_flush("save model...")
#         torch.save(best_state, '../datasets/models/baseline_LSTM.pth')
    epoch_begin = time()
    if training_termination(valid_result):
        print_flush("early stop at [%d] epoch!" % (epoch+1))
        break

In [11]:
# 10折
for i in range(1, 11):
    print('fold ', i)
    TEXT = data.Field(sequential=True, use_vocab=True, eos_token='<EOS>', init_token='<BOS>',pad_token='<PAD>', 
                      batch_first=True, tokenize=tokenizer)
    LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)
    tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                     ("txt1", TEXT), ("txt2", TEXT),
                     ("label", LABEL)]
    train = data.TabularDataset(path='../datasets/KFold/train_fold_'+str(i)+'.csv', format='csv', skip_header=True, fields=tv_datafields)
    valid = data.TabularDataset(path='../datasets/KFold/valid_fold_'+str(i)+'.csv', format='csv', skip_header=True, fields=tv_datafields)
    TEXT.build_vocab(train, valid, min_freq=3)
    matrix = wordlist_to_matrix('../datasets/pretrain_embedding/pretrain_full_emb.txt', TEXT.vocab.itos, DEVICE)
    print_flush('Building vocabulary Finished.')
    train_iter = data.BucketIterator(dataset=train, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.txt1) + len(x.txt2), shuffle=False, device=-1, repeat=False)
    valid_iter = data.Iterator(dataset=valid, batch_size=BATCH_SIZE, device=-1, shuffle=False, repeat=False)
    train_dl = BatchWrapper(train_iter, ["txt1", "txt2", "label"])
    valid_dl = BatchWrapper(valid_iter, ["txt1", "txt2", "label"])
    print_flush('prepare data done!')
    model = LSTM_INTERACTIVE_ATTENTION(TEXT.vocab, EMBED_DIM, HIDDEN_DIM, DEEP_LAYERS, matrix)
    model.to(DEVICE)
    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    criterion = nn.NLLLoss(weight=torch.tensor(CLASS_WEIGHT).float().to(DEVICE))
    optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)
    
    print_every = 50
    best_state = None
    max_metric = 0
    valid_result = []
    valid_iter.create_batches()
    valid_batch_num = len(list(valid_iter.batches))
    print_flush('start train...')
    for epoch in range(EPOCHES):
        for param_group in optimizer.param_groups:
            print('learning rate: %.6f'% param_group['lr'])
        epoch_begin = time()
        total_loss = 0.0
        train_iter.init_epoch()
        batch_count = 0
        batch_begin_time = time()
        for text1, text2, label in train_dl:
            model.train()
            text1 = text1.to(DEVICE)
            text2 = text2.to(DEVICE)
            label = label.to(DEVICE)
            x1_mask, x2_mask = get_mask(text1, text2)
            y_pred = model(text1, text2, x1_mask, x2_mask)
            loss = criterion(y_pred, label)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batch_count += 1
            if batch_count % print_every == 0:
                metric = evaluate(model.eval(), text1, text2, x1_mask, x2_mask, label)
                print_flush('[%d %d] loss: %.6f metric: %.6f time: %.1f s' %
                      (epoch + 1, batch_count, total_loss / print_every, metric, time() - batch_begin_time))
                total_loss = 0.0
                batch_begin_time = time()
    #     scheduler.step()
        print_flush("Evaluating....")
        loss, (acc, Precision, Recall, F1) = predict(model, valid_dl, criterion, DEVICE)
        valid_result.append(F1)
        print_flush('*'*60)
        print_flush('[epoch %d]. loss: %.6f acc: %.6f f1: %.6f time: %.1f s'%(epoch+1, loss/valid_batch_num, acc, F1, time()-epoch_begin))
        print_flush('*'*60)
        if F1 > max_metric:
            best_state = model.state_dict()
            max_metric = F1
            print_flush("save model...")
            torch.save(best_state, '../datasets/models/KFold/ESIM_fold_'+str(i)+'%.4f'%(F1)+'.pth')
        epoch_begin = time()
        if training_termination(valid_result):
            print_flush("early stop at [%d] epoch!" % (epoch+1))
            break


fold  1
Load embedding finished.
Total words count: 1517, oov count: 8.
Building vocabulary Finished.
prepare data done!
start train...
learning rate: 0.000950


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1 50] loss: 0.688105 metric: 0.379747 time: 9.8 s
[1 100] loss: 0.674363 metric: 0.476562 time: 9.7 s
[1 150] loss: 0.628048 metric: 0.196078 time: 10.2 s
[1 200] loss: 0.612377 metric: 0.450867 time: 9.9 s
[1 250] loss: 0.594764 metric: 0.428571 time: 10.1 s
[1 300] loss: 0.564387 metric: 0.500000 time: 10.1 s
[1 350] loss: 0.567959 metric: 0.477876 time: 10.0 s
Evaluating....
************************************************************
[epoch 1]. loss: 0.539375 acc: 0.756928 f1: 0.495851 time: 74.4 s
************************************************************
save model...
learning rate: 0.000950
[2 50] loss: 0.517073 metric: 0.641509 time: 9.9 s
[2 100] loss: 0.483164 metric: 0.604651 time: 9.8 s
[2 150] loss: 0.472346 metric: 0.542056 time: 10.3 s
[2 200] loss: 0.497346 metric: 0.636364 time: 9.9 s
[2 250] loss: 0.489188 metric: 0.564706 time: 10.2 s
[2 300] loss: 0.478531 metric: 0.600000 time: 10.2 s
[2 350] loss: 0.494633 metric: 0.580000 time: 10.3 s
Evaluating....
**********

Evaluating....
************************************************************
[epoch 1]. loss: 0.522769 acc: 0.735265 f1: 0.514408 time: 74.9 s
************************************************************
save model...
learning rate: 0.000950
[2 50] loss: 0.503221 metric: 0.655367 time: 9.7 s
[2 100] loss: 0.476953 metric: 0.595506 time: 9.7 s
[2 150] loss: 0.470129 metric: 0.534884 time: 10.3 s
[2 200] loss: 0.497039 metric: 0.632353 time: 10.0 s
[2 250] loss: 0.486813 metric: 0.625000 time: 10.1 s
[2 300] loss: 0.481175 metric: 0.529915 time: 10.2 s
[2 350] loss: 0.489637 metric: 0.557692 time: 10.2 s
Evaluating....
************************************************************
[epoch 2]. loss: 0.496458 acc: 0.816062 f1: 0.569536 time: 74.9 s
************************************************************
save model...
learning rate: 0.000950
[3 50] loss: 0.426019 metric: 0.650307 time: 9.8 s
[3 100] loss: 0.411151 metric: 0.670659 time: 9.7 s
[3 150] loss: 0.410940 metric: 0.534884 time: 1

[5 150] loss: 0.319370 metric: 0.736842 time: 10.5 s
[5 200] loss: 0.363603 metric: 0.750000 time: 10.0 s
[5 250] loss: 0.340972 metric: 0.783505 time: 10.2 s
[5 300] loss: 0.350628 metric: 0.741379 time: 10.1 s
[5 350] loss: 0.358264 metric: 0.689655 time: 10.1 s
Evaluating....
************************************************************
[epoch 5]. loss: 0.668482 acc: 0.820746 f1: 0.565824 time: 74.7 s
************************************************************
learning rate: 0.000950
[6 50] loss: 0.295466 metric: 0.784314 time: 9.6 s
[6 100] loss: 0.264316 metric: 0.772414 time: 9.7 s
[6 150] loss: 0.285248 metric: 0.744681 time: 10.5 s
[6 200] loss: 0.336778 metric: 0.720000 time: 10.0 s
[6 250] loss: 0.307273 metric: 0.795699 time: 10.2 s
[6 300] loss: 0.314175 metric: 0.784314 time: 10.1 s
[6 350] loss: 0.337346 metric: 0.708333 time: 10.1 s
Evaluating....
************************************************************
[epoch 6]. loss: 0.880056 acc: 0.829723 f1: 0.556544 time: 74.7 

[10 350] loss: 0.198630 metric: 0.795699 time: 10.0 s
Evaluating....
************************************************************
[epoch 10]. loss: 0.780588 acc: 0.750073 f1: 0.532579 time: 74.9 s
************************************************************
learning rate: 0.000950
[11 50] loss: 0.194942 metric: 0.777070 time: 9.8 s
[11 100] loss: 0.230831 metric: 0.811594 time: 9.7 s
[11 150] loss: 0.190757 metric: 0.886364 time: 10.3 s
[11 200] loss: 0.189501 metric: 0.847458 time: 10.0 s
[11 250] loss: 0.178154 metric: 0.930233 time: 10.2 s
[11 300] loss: 0.198043 metric: 0.891089 time: 10.1 s
[11 350] loss: 0.179800 metric: 0.672727 time: 10.1 s
Evaluating....
************************************************************
[epoch 11]. loss: 1.276284 acc: 0.838684 f1: 0.566256 time: 74.6 s
************************************************************
learning rate: 0.000950
[12 50] loss: 0.185847 metric: 0.916031 time: 9.8 s
[12 100] loss: 0.207054 metric: 0.853659 time: 9.8 s
[12 150] l